# AgriMarket

## 농수산물


- Data Source : [농식품 빅데이터 거래소](https://kadx.co.kr/product/detail/94534d90-595d-11eb-acdd-9560621d9652)

- Collection Method : Download

- Data Format : CSV

### 데이터 설명

- 일반농산물 도매 69품목 116품종, 소매 90품목 143품종 및 친환경농산물 38품목 44종 대상 가격 자료


## 소비자 물가지수 


- Data Source : [국가통계포털](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_1J20112&conn_path=I2)

- Collection Method : Download

- Data Format : CSV

### 데이터 설명

- 농산물 중 필수재라고 판단되는 곡물,채소 포함

- 사치재인 과실, 기타 농산물 제외

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import koreanize_matplotlib
%config InlineBackend.figure_format = 'retina'

root_path = "../"
data_path = f"{root_path}data"

col_detail={
'prce_reg_ymd' : '가격등록일자',
'mrkt_esnt_no' : '시장고유번호',
'mrkt_code' : '시장코드',
'mrkt_nm' : '시장명',
'ct_code' : '시도코드',
'ctnp_nm' : '시도명',
'sggu_code' : '시군구코드',
'sggu_nm' : '시군구명',
'pdlt_code' : '품목코드',
'pdlt_nm' : '품목명',
'spcs_code' : '품종코드',
'spcs_nm' : '품종명',
'wsrt_exmn_se_code' : '도소매조사구분코드',
'exmn_se_nm' : '조사구분명',
'bulk_grad_code' : '산물등급코드',
'bulk_grad_nm' : '산물등급명',
'pdlt_prce' : '품목가격',
'mtc_smt_unit_mg' : '산지출하단위크기',
'mtc_smt_unit_nm' : '산지출하단위명',
'whsl_smt_unit_mg' : '도매출하단위크기',
'whsl_smt_unit_nm' : '도매출하단위명',
'rtsl_smt_unit_mg' : '소매출하단위크기',
'rtsl_smt_unit_nm' : '소매출하단위명',
'evfd_fmpd_smt_unit_mg' : '친환경농산물출하단위크기',
'evfd_fmpd_smt_unit_nm' : '친환경농산물출하단위명',
'dcnt_prce_yn' : '할인가격여부',
'etl_ldg_dt' : 'ETL적재일시'
}

c:\Users\sacer\miniconda3\envs\asd1\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# 파일 호출, 변수명 및 불러오기 경로는 자신에게 맞게 변경해주시면 됩니다.

In [2]:
import os
files_path = f'{data_path}/TB_TAT_KAMIS_PRC_MK_TB/'
files = os.listdir(files_path)
files.remove('.DS_Store')
print(files)

ValueError: list.remove(x): x not in list

In [3]:
df = pd.DataFrame()
for file in files:
    df_sub = pd.read_csv(f'{files_path}/{file}', encoding='cp949', low_memory=False)
    df_sub = df_sub[['PRCE_REG_YMD', 'CTNP_NM', 'PDLT_NM', 'EXMN_SE_NM', 'BULK_GRAD_NM', 'PDLT_PRCE']]
    df_sub = df_sub[~df_sub['PDLT_NM'].isna()]
    df_sub.columns=['YMD', 'Region', 'Product', 'WR', 'Grade', 'Price']
    df = pd.concat([df, df_sub])

# 1. 년·월·일 에서 연·월 로 변경
# 2. Y(년), M(월) 컬럼 생성

In [4]:
df['YMD'] = df['YMD'].astype(str)
df['YMD'] = df['YMD'].map(lambda x:x[:6])

df['Y'] = df['YMD'].map(lambda x:x[:4]).astype('int16')
df['M'] = df['YMD'].map(lambda x:x[4:]).astype('int8')

In [5]:
# 20년 데이터만 있는 품목 제외

df = df[~df['Product'].isin(['즉석밥', '두부', '김치', '고추장', '된장', '간장', '맛김(조미김)', '콩나물'])]

In [6]:
# Wholesale : 도매
df_w = df[df['WR']=='도매']

In [7]:
# Retail : 소매
df_r = df[df['WR']=='소매']

In [8]:
product_w = df_w['Product'].unique()
product_w = product_w.tolist()

In [9]:
product_r = df_r['Product'].unique()
product_r = product_r.tolist()

In [3]:
df

NameError: name 'df' is not defined

# 물가지수 파일과 겹치는 품목 확인

In [11]:
df_1 = pd.read_csv(f'{data_path}/소비자물가지수_품목별_연도별_월별_filtered.csv')

In [12]:
df_1['품목별'].unique()

array(['총지수', '농축수산물', '\u3000\u3000\u3000(농산물)곡물',
       '\u3000\u3000\u3000\u3000\u3000\u3000쌀',
       '\u3000\u3000\u3000\u3000\u3000\u3000현미',
       '\u3000\u3000\u3000\u3000\u3000\u3000찹쌀',
       '\u3000\u3000\u3000\u3000\u3000\u3000보리쌀',
       '\u3000\u3000\u3000\u3000\u3000\u3000콩',
       '\u3000\u3000\u3000\u3000\u3000\u3000땅콩',
       '\u3000\u3000\u3000\u3000\u3000\u3000혼식곡',
       '\u3000\u3000\u3000(농산물)채소',
       '\u3000\u3000\u3000\u3000\u3000\u3000배추',
       '\u3000\u3000\u3000\u3000\u3000\u3000상추',
       '\u3000\u3000\u3000\u3000\u3000\u3000시금치',
       '\u3000\u3000\u3000\u3000\u3000\u3000양배추',
       '\u3000\u3000\u3000\u3000\u3000\u3000미나리',
       '\u3000\u3000\u3000\u3000\u3000\u3000깻잎',
       '\u3000\u3000\u3000\u3000\u3000\u3000부추',
       '\u3000\u3000\u3000\u3000\u3000\u3000무',
       '\u3000\u3000\u3000\u3000\u3000\u3000열무',
       '\u3000\u3000\u3000\u3000\u3000\u3000당근',
       '\u3000\u3000\u3000\u3000\u3000\u3000감자',
       '\u3000\u3000\u3000\u

In [13]:
# replace로 \u3000 값이 사라지지 않는다.. 이유는 모르겠음
df_1['품목별'] = df_1['품목별'].replace('\u3000', '')

In [14]:
df_1['품목별'] = df_1['품목별'].map(lambda x:x.strip(('\u3000')))

In [15]:
df['Product'].unique()

array(['단감', '바나나', '참다래', '파인애플', '오렌지', '방울토마토', '고등어', '갈치', '명태',
       '물오징어', '건멸치', '북어', '건오징어', '김', '건미역', '굴', '고구마', '감자', '배추',
       '양배추', '시금치', '상추', '얼갈이배추', '수박', '오이', '호박', '토마토', '딸기', '무',
       '당근', '열무', '건고추', '풋고추', '붉은고추', '피마늘', '양파', '파', '생강', '미나리',
       '깻잎', '피망', '파프리카', '멜론', '느타리버섯', '팽이버섯', '새송이버섯', '사과', '배',
       '레몬', '망고', '전복', '새우', '깐마늘(국산)', '감귤', '포도', '쌀', '찹쌀', '콩', '팥',
       '녹두', '메밀', '참깨', '들깨', '땅콩', '쇠고기', '돼지고기', '닭고기', '계란', '수입조기',
       '호두', '아몬드', '건포도', '건블루베리', '고춧가루', '멸치액젓', '갓', '새우젓', '굵은소금',
       '꽁치', '우유', '부추', '참외', '체리', '복숭아', '가지', '자몽', '깐마늘(수입)', '브로콜리',
       '청경채', '케일', '양송이버섯', '표고버섯', '귀리', '보리', '수수', '율무', '양상추', '혼합곡'],
      dtype=object)

# 농수산물 데이터와 주요경제지표 합치기

In [16]:
file = f"{data_path}/주요경제지표_20052021.xlsx"
df_mei = pd.read_excel(file, engine="openpyxl")
print(df_mei.shape)
display(df_mei.head(df_mei.shape[0]))

(17, 11)


/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,연도,생산자물가지수(2010=100),소비자물가지수(2015=100),경상수지(백만US$),자본수지(백만US$),외환보유액(백만US$),수출액(백만US$),수입액(백만US$),환율(원/US$),실업률(%),콜금리(연%)
0,2021,110.64,108.050,88302.7,-154.3,463118.360,644400.370,615093.450,1144.42,3.7,0.61
1,2020,104.01,105.420,75275.7,-386.3,443098.120,512788.730,467548.715,1180.05,3.9,0.70
2,2019,104.48,104.850,59670.1,-169.3,408816.146,542232.610,503342.947,1157.80,3.8,1.59
3,2018,104.46,104.450,76408.5,316.7,403694.264,604859.657,535202.428,1118.10,3.8,1.52
4,2017,102.53,102.930,78460.0,-26.8,389266.683,573694.421,478478.296,1071.40,3.7,1.26
5,2016,99.11,100.970,98677.4,-46.2,371101.605,495425.934,406192.887,1208.50,3.7,1.34
6,2015,100.95,100.000,105939.6,-60.2,367961.865,526756.498,436498.973,1172.00,3.6,1.65
7,2014,105.17,99.298,84374.0,-8.9,363592.748,572664.602,525514.506,1099.20,3.5,2.34
8,2013,105.73,98.048,81148.2,-27.0,346459.569,559632.400,515585.500,1055.30,3.1,2.59
9,2012,107.45,96.789,50835.0,-41.7,326968.393,547869.800,519584.500,1071.10,3.2,3.08


In [17]:
df_mei = df_mei.rename(columns={"연도" : "Y"})

In [18]:
pd.concat([df, df_mei], axis=0)

,YMD,Region,Product,WR,Grade,Price,Y,M,생산자물가지수(2010=100),소비자물가지수(2015=100),경상수지(백만US$),자본수지(백만US$),외환보유액(백만US$),수출액(백만US$),수입액(백만US$),환율(원/US$),실업률(%),콜금리(연%)
0,201901,서울,단감,도매,상품,33000.0,2019,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201901,서울,단감,도매,중품,28000.0,2019,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201901,서울,바나나,도매,상품,22000.0,2019,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201901,서울,바나나,도매,중품,17000.0,2019,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201901,서울,참다래,도매,상품,35000.0,2019,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12,NaN,NaN,NaN,NaN,NaN,NaN,2009,NaN,96.33,88.452,33593.3,-69.6,269994.736,363533.400,323084.400,1167.6,3.6,1.98
13,NaN,NaN,NaN,NaN,NaN,NaN,2008,NaN,96.53,86.079,3189.7,26.4,201223.413,422007.300,435274.700,1257.5,3.2,4.78
14,NaN,NaN,NaN,NaN,NaN,NaN,2007,NaN,88.93,82.235,11794.5,5.7,262224.070,371489.000,356845.800,938.2,3.2,4.77
15,NaN,NaN,NaN,NaN,NaN,NaN,2006,NaN,87.67,80.202,3569.2,-69.0,238956.116,325464.849,309382.632,929.6,3.5,4.19
